# 비선형 최소 자승법 (nonlinear least squares problem)

### nonlinear optimization과의 관계
- 이전에 비선형 최적화 문제들이 주어진 함수 f : R^n -> R가 x_0에서부터 시작하여,  
  f(x_0) > f(x_1) > f(x_2) ...  
  를 만족하는 파라미터 x를 찾아가는 방법이었다.
- 이제 다루고자 하는 문제는 f : R^n -> R^m을 만족하는 다변수 함수에 대해서  
  f(x_0) > f(x_1) > f(x_2) ...  -> q = (q1, q2,..., qm)^T  
  를 만족하는 파라미터를 찾는 문제이다.
- f(x) = q를 만족하는 x를 만족하는 문제는 f_hat(x) = f(x) - q = 0을 만족하는 문제로 생각할 수 있다.  
  이 문제는 ||f_hat(x)|| = 0에 가까워지는 문제라고 볼 수 있으며, 이 문제는 미분식의 간단함을 위해서  0.5 ||f_hat(x)||^2라는  
  문제로 변형되어 사용된다.
- 다시 정리하면 아래와 같다.  
  F(x) = 0.5 ∑^m (f_i(x))^2 = 0.5 ||f(x)||^2 = 0.5 f(x)^T*f(x) 이고  
  F(x)가 최소인 x*의 값을 구하는 방법으로 비선형 최소자승법이라고 한다.  
  ※ 여기서 m은 보통 샘플의 수를 나타낸다.

## Gauss Newton 탐색방법

### 특징
- Levenberg marquardt 탐색 방법의 기본방법이고, 충분히 작은 ||h||에 대하여  
  l(h) = f(x + h) = f(x) + J(x)h 라는 근사식에서 시작한다.

### 유도
- F(x + h)의 근사식은 다음과 같다.  
  F(x + h) = L(h) = 0.5 l(h)^T l(h)
  = 0.5 * f^t(x) * f(x) + h^t * J^t(x) * f(x) + 0.5 * h^t * J^t(x) * J(x) * h  
  = F(x) + h^t * J^t(x) * f(x) + 0.5 * h^t * J^t(x) * J(x) * h
- L(h)가 최소가 되는 h를 찾으면,  
  L'(h) = J^t(x) * f(x) + J^t(x) * J(x) * h  
  L''(h) = J^t(x) * J(x) 이고
- 최소값을 가지는 L'(h_gn) = 0에서  
  J^t(x) * J(x) * h_gn = - J^t(x) * f(x)  
  이다.
- 만약 J(x)의 열벡터가 일차 독립이면, L''(h) = J^t(x)J(x)는 양의 확정행렬이고, 전역 최소값이 존재한다.
- 여기서 구해진 h_gn 은 하강 방향벡터이자. L(h)의 유일한 전역 최소자이다.

### 업데이트 방법
1. J^t(x) * J(x) * h_gn = - J^t(x) * f(x)에서 h_gn을 구한다.
2. x_{k+1} = x_k + a*h_gn 로 업데이트한다.  
   ※ a는 step size로 직선탐색 알고리즘을 이용한다.

### 예제 결과
1. 예제 1과 같이 F(x, y)가 컨벡스 함수가 아니라면 잘 수렴하지 못한다. 이것은 Levenberg maquardt 탐색방법도 비슷하다.
2. 예제 2와 같이 F(x, y)가 컨벡스 함수면 빠르게 수렴함.

In [39]:
import numpy as np
# 예제 1 가상의 a,b를 추론해보자. 
## y = a * cos(b*x) + b sin (a*x)

sample_size = 100
sample_x = 2* np.pi * np.random.rand(sample_size, 1)
noise = 0#0.01 * np.random.rand(sample_size, 1)

a = 100
b = 102

sample_y = a * np.cos( b * sample_x) + b * np.sin(a * sample_x)
noised_sample_y = sample_y + noise

iterations = 1000
init_a = 90
init_b = 102.6
step_size = 1
def f(x, arg_a, arg_b):
    return arg_a * np.cos( arg_b * x) + arg_b * np.sin(arg_a * x)

def gradient_f(x, arg_a, arg_b):
    gradient_a = np.cos( arg_b * x) + arg_b * np.cos(arg_a * x) * x
    gradient_b = -1.0 * arg_a * np.sin( arg_b * x) * x + np.sin(arg_a * x)
    return np.hstack((gradient_a, gradient_b))

def F(x, y, arg_a, arg_b):
    residual = y - f(x, arg_a, arg_b)
    return (0.5 * residual.T * residual)[0, 0]

updated_a = init_a
updated_b = init_b
for iter in range(iterations):
    jacobian = gradient_f(sample_x, updated_a, updated_b)
    residual = noised_sample_y - f(sample_x, updated_a, updated_b)
    j_t_j = jacobian.T @ jacobian
    minus_j_t_f = jacobian.T @ residual
    cost = F(sample_x, noised_sample_y, updated_a, updated_b)
    if iter % 100 ==0:
        print("Iter : ",iter,", Cost : ", cost, ", Parameter a : ", updated_a, ", b : ", updated_b)
    a_b_change = np.linalg.solve(j_t_j, minus_j_t_f)
    updated_a += step_size * a_b_change[0, 0]
    updated_b += step_size * a_b_change[1, 0]

Iter :  0 , Cost :  8.768166690204603e-05 , Parameter a :  90 , b :  102.6
Iter :  100 , Cost :  860.3349548020047 , Parameter a :  90.11534484984689 , b :  102.0235493928277
Iter :  200 , Cost :  860.3349548020047 , Parameter a :  90.11534484984689 , b :  102.0235493928277
Iter :  300 , Cost :  860.3349548020047 , Parameter a :  90.11534484984689 , b :  102.0235493928277
Iter :  400 , Cost :  860.3349548020047 , Parameter a :  90.11534484984689 , b :  102.0235493928277
Iter :  500 , Cost :  860.3349548020047 , Parameter a :  90.11534484984689 , b :  102.0235493928277
Iter :  600 , Cost :  860.3349548020047 , Parameter a :  90.11534484984689 , b :  102.0235493928277
Iter :  700 , Cost :  860.3349548020047 , Parameter a :  90.11534484984689 , b :  102.0235493928277
Iter :  800 , Cost :  860.3349548020047 , Parameter a :  90.11534484984689 , b :  102.0235493928277
Iter :  900 , Cost :  860.3349548020047 , Parameter a :  90.11534484984689 , b :  102.0235493928277


In [38]:
import numpy as np
# 예제 2 가상의 a, b를 추론해보자.
## f(x) = a * x^2 + b

sample_size = 100
sample_x = np.random.rand(sample_size, 1)
noise = 0.1 * np.random.rand(sample_size, 1)

a = 100
b = 102

sample_y = a * sample_x * sample_x + b * np.ones((sample_size, 1))
noised_sample_y = sample_y + noise

iterations = 3
init_a = 90
init_b = 102.6
step_size = 1
def f(x, arg_a, arg_b):
    return arg_a * x * x + arg_b * np.ones(x.shape)

def gradient_f(x, arg_a, arg_b):
    gradient_a = x * x
    gradient_b = np.ones(x.shape)
    return np.hstack((gradient_a, gradient_b))

def F(x, y, arg_a, arg_b):
    residual = y - f(x, arg_a, arg_b)
    return (0.5 * residual.T * residual)[0, 0]

updated_a = init_a
updated_b = init_b
for iter in range(iterations):
    jacobian = gradient_f(sample_x, updated_a, updated_b)
    residual = noised_sample_y - f(sample_x, updated_a, updated_b)
    j_t_j = jacobian.T @ jacobian
    minus_j_t_f = jacobian.T @ residual
    cost = F(sample_x, noised_sample_y, updated_a, updated_b)
    if iter % 1 ==0:
        print("Iter : ",iter,", Cost : ", cost, ", Parameter a : ", updated_a, ", b : ", updated_b)
    a_b_change = np.linalg.solve(j_t_j, minus_j_t_f)
    updated_a += step_size * a_b_change[0, 0]
    updated_b += step_size * a_b_change[1, 0]

Iter :  0 , Cost :  32.78390600303072 , Parameter a :  90 , b :  102.6
Iter :  1 , Cost :  0.00023782741480193515 , Parameter a :  100.00396742995875 , b :  102.04664472714445
Iter :  2 , Cost :  0.00023782741480193515 , Parameter a :  100.00396742995875 , b :  102.04664472714445


## Levenberg-Marquardt

### 설명과 유도
- Gradient descent 탐색 방법과 Newton 탐색 방법의 혼합형이다.
- 최소자로부터 상대적으로 먼 거리 점에서 시작할 때는 Gradient descent 탐색방법을 이용하고 최소자 금방에서는 수렴속도가 좋은 newton 탐색 방법을 사용한다.
- 다음과 같이 damping 인자를 추가한다.  
  (J^t(x) * J(x) + mu * I) * h_lm = - J^t(x) * f(x)
- mu가 작으면, 기존 newton method랑 동일해지고, mu가 상당히 크다면, h_lm = -J^t(x) * f(x)/mu 가 된다.

### mu를 어떻게 업데이트 해야하는가?
- 이득 비율(gain ratio)를 계산해서, 매번 mu를 업데이트 해야한다.
- p = (F(x) - F(x + h_dn))/(L(0) - L(h_dn)), where L(h) = F(x) + h^t * J^t(x) * f(x) + 0.5 * h^t * J^t(x) * J(x) * h
- 위의 식에서 p값이 1에 가깝게 나오면, mu를 줄이고 반대인 경우는 mu값을 크게 한다.
- p > 0.75, mu = mu/3
- p < 0.25, mu = mu*2
- 추가로 p가 0.75나 0.25 근처 값에서 불연속적으로 변하는 피하기 위해서, 다음과 같이 수정한다.
- if p > 0 then mu = mu * max{1/3, 1-(2 * p - 1)^3} else mu = mu*2

### 정지조건은 어떻게 설정하는가?
- ||▽f(x)||_{inf} <= epsilon_1 또는 ||h_dn|| <= epsilon_2(epsilon_2 + ||x||) 사용한다.
- epsilon_1 = 10^-8, epsilon_2 = 10^-8를 많이 사용한다.

### 예제 결과
- 예제 3도 마찬가지로 잘 수렴하지 못하는 것을 확인할 수 있다.
- 예제 4도 마찬가지로 잘 수렴한다.
- 이러한 결과를 봤을 때, 여전히 convex하지 못한 문제의 경우 수렴이 잘못한다는것을 알았다.

In [49]:
import numpy as np
# 예제 3 가상의 a,b를 추론해보자.(예제 1과 같음)
## y = a * cos(b*x) + b sin (a*x)

sample_size = 100
sample_x = 2* np.pi * np.random.rand(sample_size, 1)
noise = 0#0.01 * np.random.rand(sample_size, 1)

a = 100
b = 102

sample_y = a * np.cos( b * sample_x) + b * np.sin(a * sample_x)
noised_sample_y = sample_y + noise

iterations = 1000
init_a = 90
init_b = 102.6
init_mu = 1
epsilon = 1.0/10.0**8
step_size = 1
def f(x, arg_a, arg_b):
    return arg_a * np.cos( arg_b * x) + arg_b * np.sin(arg_a * x)

def gradient_f(x, arg_a, arg_b):
    gradient_a = np.cos( arg_b * x) + arg_b * np.cos(arg_a * x) * x
    gradient_b = -1.0 * arg_a * np.sin( arg_b * x) * x + np.sin(arg_a * x)
    return np.hstack((gradient_a, gradient_b))

def F(x, y, arg_a, arg_b):
    residual = y - f(x, arg_a, arg_b)
    return (0.5 * residual.T * residual)[0, 0]

def L(x, y, arg_a, arg_b, h):
    first_term = F(x, y, arg_a, arg_b)
    second_term = (h.T @ gradient_f(x, arg_a, arg_b).T @ f(x, arg_a, arg_b))[0, 0]
    third_term = (0.5 * h.T @ gradient_f(x, arg_a, arg_b).T @ gradient_f(x, arg_a, arg_b) @ h)[0, 0]
    return first_term + second_term + third_term

updated_a = init_a
updated_b = init_b
mu = init_mu
for iter in range(iterations):
    jacobian = gradient_f(sample_x, updated_a, updated_b)
    residual = noised_sample_y - f(sample_x, updated_a, updated_b)
    j_t_j_plus_mu = jacobian.T @ jacobian + mu * np.eye(2)
    minus_j_t_f = jacobian.T @ residual
    cost = F(sample_x, noised_sample_y, updated_a, updated_b)
    a_b_change = np.linalg.solve(j_t_j_plus_mu, minus_j_t_f)
    p = (F(sample_x, sample_y, updated_a, updated_b) -\
        F(sample_x, sample_y, updated_a + step_size * a_b_change[0, 0], updated_b + step_size * a_b_change[0, 0])) / \
        (L(sample_x, sample_y, updated_a, updated_b, np.zeros((2, 1))) -\
        L(sample_x, sample_y, updated_a, updated_b, a_b_change))
    updated_a += step_size * a_b_change[0, 0]
    updated_b += step_size * a_b_change[1, 0]
    if p > 0:
        mu = mu/3.0 if mu/3.0 > 1-(2*p -1)**2 else 1-(2*p -1)**2
    else :
        mu = mu  * 2
    if iter % 1 ==0:
        print("Iter : ",iter,", Cost : ", cost, ", Parameter a : ", updated_a, ", b : ", updated_b)
    if np.linalg.norm(jacobian, np.inf) <= epsilon:
        print("수렴완료")
        break

Iter :  0 , Cost :  140.55404456352966 , Parameter a :  90.05905773843439 , b :  102.56530471104915
Iter :  1 , Cost :  89.7947547148015 , Parameter a :  90.12794691984884 , b :  102.47807459966371
Iter :  2 , Cost :  32.74013909861146 , Parameter a :  90.18770868034524 , b :  102.30241486288317
Iter :  3 , Cost :  0.7869829860250208 , Parameter a :  90.20364907666371 , b :  102.04030505289688
Iter :  4 , Cost :  7.837918442884727 , Parameter a :  90.20722102311498 , b :  102.00662985357114
Iter :  5 , Cost :  10.269199778792295 , Parameter a :  90.2096830443627 , b :  102.01862814693763
Iter :  6 , Cost :  10.11800461151995 , Parameter a :  90.21124579872088 , b :  102.01446531737659
Iter :  7 , Cost :  10.664343420085244 , Parameter a :  90.21224043337627 , b :  102.0159199702107
Iter :  8 , Cost :  10.788935454492394 , Parameter a :  90.21287224546211 , b :  102.0154108354797
Iter :  9 , Cost :  10.948965783072842 , Parameter a :  90.2132728943277 , b :  102.01558681701927
Iter :  1

C:\Users\Jeon\AppData\Local\Temp\ipykernel_4600\2021286778.py:49: RuntimeWarning: invalid value encountered in scalar divide
  p = (F(sample_x, sample_y, updated_a, updated_b) -\


In [54]:
import numpy as np
# 예제 4 가상의 a,b를 추론해보자.(예제 2와 같음)
## f(x) = a * x^2 + b

sample_size = 100
sample_x = np.random.rand(sample_size, 1)
noise = 0.1 * np.random.rand(sample_size, 1)

a = 100
b = 102

sample_y = a * sample_x * sample_x + b * np.ones((sample_size, 1))
noised_sample_y = sample_y + noise

iterations = 3
init_a = 40
init_b = 30
init_mu = 1
epsilon = 1.0/10.0**8
step_size = 1
def f(x, arg_a, arg_b):
    return arg_a * x * x + arg_b * np.ones(x.shape)

def gradient_f(x, arg_a, arg_b):
    gradient_a = x * x
    gradient_b = np.ones(x.shape)
    return np.hstack((gradient_a, gradient_b))

def F(x, y, arg_a, arg_b):
    residual = y - f(x, arg_a, arg_b)
    return (0.5 * residual.T * residual)[0, 0]

def L(x, y, arg_a, arg_b, h):
    first_term = F(x, y, arg_a, arg_b)
    second_term = (h.T @ gradient_f(x, arg_a, arg_b).T @ f(x, arg_a, arg_b))[0, 0]
    third_term = (0.5 * h.T @ gradient_f(x, arg_a, arg_b).T @ gradient_f(x, arg_a, arg_b) @ h)[0, 0]
    return first_term + second_term + third_term

updated_a = init_a
updated_b = init_b
mu = init_mu
for iter in range(iterations):
    jacobian = gradient_f(sample_x, updated_a, updated_b)
    residual = noised_sample_y - f(sample_x, updated_a, updated_b)
    j_t_j_plus_mu = jacobian.T @ jacobian + mu * np.eye(2)
    minus_j_t_f = jacobian.T @ residual
    cost = F(sample_x, noised_sample_y, updated_a, updated_b)
    a_b_change = np.linalg.solve(j_t_j_plus_mu, minus_j_t_f)
    p = (F(sample_x, sample_y, updated_a, updated_b) -\
        F(sample_x, sample_y, updated_a + step_size * a_b_change[0, 0], updated_b + step_size * a_b_change[0, 0])) / \
        (L(sample_x, sample_y, updated_a, updated_b, np.zeros((2, 1))) -\
        L(sample_x, sample_y, updated_a, updated_b, a_b_change))
    updated_a += step_size * a_b_change[0, 0]
    updated_b += step_size * a_b_change[1, 0]
    if p > 0:
        mu = mu/3.0 if mu/3.0 > 1-(2*p -1)**2 else 1-(2*p -1)**2
    else :
        mu = mu  * 2
    if iter % 1 ==0:
        print("Iter : ",iter,", Cost : ", cost, ", Parameter a : ", updated_a, ", b : ", updated_b)
    if np.linalg.norm(jacobian, np.inf) <= epsilon:
        print("수렴완료")
        break

Iter :  0 , Cost :  6137.483760095832 , Parameter a :  96.63028750944837 , b :  102.58611410907068
Iter :  1 , Cost :  1.4175470734315632 , Parameter a :  99.35081660208111 , b :  102.29826647686185
Iter :  2 , Cost :  0.023559890147092186 , Parameter a :  99.94623491626972 , b :  102.07090376388517
